In [2]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [3]:
if not os.path.exists('log/'):
    os.makedirs('log/')

# Launcher

This notebook automatically launches all operations for a complete model run. One can decide to skip network preparation steps, as they take multiple hours for the region of Germany and the networks are readily included in the repository.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `parameters.xls` file. This file also includes scenarios.

In [4]:
kwargs = {'workers':4, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

In [5]:
scenarios = ['base']

In [6]:
import time
start = time.time()

# Network preparation

Execute all `prep1` and `prep2` steps.

In [7]:
# Network generation
parallel_call_notebook('prep10_zones.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebooks(
    ('prep11_road.ipynb', scenarios),
    ('prep12_rail_coach.ipynb', scenarios),
    ('prep14_bus.ipynb', scenarios),
    **kwargs)
parallel_call_notebook('prep13_air.ipynb', arg_list=scenarios, **kwargs)

0 prep12_rail_coach.py base
1 prep14_bus.py base
subprocess **prep14_bus.py1 base** terminated with an error.
1763 seconds
0 prep13_air.py base
28 seconds


In [ ]:
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('prep15_census_distances.ipynb', arg_list=scenarios, **kwargs)

In [9]:
# Network aggregation
# First cluster stops, then aggregate trips
parallel_call_notebook('prep20_cluster.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep22_access_egress_road.ipynb', arg_list=scenarios, **kwargs)

0 prep20_cluster.py base
3073 seconds
0 prep21_aggregate_pt.py base
945 seconds
0 prep22_access_egress_road.py base
77 seconds


# Pathfinders and level of service

In [ ]:
# Pathfinders
parallel_call_notebooks(
    ('prep30_pathfinder_pt.ipynb', scenarios),
    ('prep30_pathfinder_road.ipynb', scenarios),
    **kwargs)

In [10]:
# PT LoS attributes
parallel_call_notebook('prep30_pathfinder_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep33_non-motor.ipynb', arg_list=scenarios, **kwargs)

0 prep30_pathfinder_pt.py base
3290 seconds
0 prep31_prices_pt.py base
84 seconds
0 prep33_non-motor.py base
subprocess **prep33_non-motor.py0 base** terminated with an error.
31 seconds


# Estimation

In [ ]:
# Calibration data preparation
# NOTE: only applicapble if you have access to the calibration dataset
parallel_call_notebook('cal10_input_data.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('cal11_los_columns.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal1x_validation_data.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal19_vot.ipynb', arg_list=scenarios, **kwargs)

In [11]:
# Estimation
parallel_call_notebook('cal20_estimation.ipynb', arg_list=scenarios, **kwargs)

0 cal20_estimation.py base
subprocess **cal20_estimation.py0 base** terminated with an error.
4 seconds


# Modelling

Except for the logit model step, all modelling steps require access to the OD volumes dataset.

In [ ]:
# Generation of the OD matrix
parallel_call_notebook('model_volumes.ipynb', arg_list=scenarios, **kwargs)

In [12]:
# Mode choice probabilities
parallel_call_notebook('model_logit.ipynb', arg_list=scenarios, **kwargs)

0 model_logit.py base
subprocess **model_logit.py0 base** terminated with an error.
41 seconds


In [13]:
# Assignment
parallel_call_notebooks(
    ('model_assignment.ipynb', scenarios),
    ('model_assignment_inner-zonal.ipynb', scenarios),
    **kwargs)

0 model_assignment.py base
1 model_assignment_inner-zonal.py base
subprocess **model_assignment.py0 base** terminated with an error.
330 seconds


In [14]:
# Post-processing
parallel_call_notebook('model_emissions.ipynb', arg_list=scenarios, **kwargs)

0 model_emissions.py base
210 seconds


In [ ]:
# Total execution time
end = time.time()
int(end - start)